In [1]:
import numpy as np
import json
from keras.utils.np_utils import to_categorical
import time
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [25]:
with open('Random_Smart_Losses.json') as json_file:
    training = json.load(json_file)

In [26]:
for item in [training]:
    print(len(item['moves']))
    print(len(item['boards']))

54620
54620


In [27]:
X = np.array(training['boards'])
y = to_categorical(training['moves'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

In [28]:
for i in [X_train, X_test, X_val, y_train, y_test, y_val]:
    print(len(i))

49158
4369
1093
49158
4369
1093


In [29]:
from keras.models import load_model
parial_model = load_model('smart_model_partial.h5')

In [30]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics


history = parial_model.fit(X_train,
                   y_train,
                   epochs=10,
                   batch_size = 512,
                   validation_data = (X_test, y_test))

Train on 49158 samples, validate on 4369 samples
Epoch 1/10
49158/49158 [==============================] - 5s 97us/step - loss: 1.2490 - acc: 0.4818 - val_loss: 1.2636 - val_acc: 0.4768
Epoch 2/10
49158/49158 [==============================] - 5s 96us/step - loss: 1.2456 - acc: 0.4813 - val_loss: 1.2528 - val_acc: 0.4726
Epoch 3/10
49158/49158 [==============================] - 5s 95us/step - loss: 1.2345 - acc: 0.4816 - val_loss: 1.2700 - val_acc: 0.4765
Epoch 4/10
49158/49158 [==============================] - 5s 94us/step - loss: 1.2335 - acc: 0.4843 - val_loss: 1.2604 - val_acc: 0.4749
Epoch 5/10
49158/49158 [==============================] - 5s 96us/step - loss: 1.2325 - acc: 0.4818 - val_loss: 1.2452 - val_acc: 0.4868
Epoch 6/10
49158/49158 [==============================] - 5s 95us/step - loss: 1.2315 - acc: 0.4837 - val_loss: 1.2448 - val_acc: 0.4846
Epoch 7/10
49158/49158 [==============================] - 5s 95us/step - loss: 1.2317 - acc: 0.4839 - val_loss: 1.2513 - val_acc:

In [31]:
parial_model.save('pre-trainied_loss_model.h5')

In [32]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

new_model = models.Sequential()

new_model.add(layers.Dense(729, activation = 'relu', input_shape = (9,)))
new_model.add(layers.Dense(729, activation = 'relu'))
new_model.add(layers.Dense(729, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(81, activation = 'relu'))
new_model.add(layers.Dense(9, activation = 'softmax'))



new_model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


# train the network
now = time.time()

history = new_model.fit(X_train,
                   y_train,
                   epochs=10,
                   batch_size = 512,
                   validation_data = (X_test, y_test))

print(time.time()-now)

print(new_model.evaluate(X_val, y_val))

Train on 49158 samples, validate on 4369 samples
Epoch 1/10
49158/49158 [==============================] - 5s 101us/step - loss: 1.4658 - acc: 0.4244 - val_loss: 1.5264 - val_acc: 0.4372
Epoch 2/10
49158/49158 [==============================] - 5s 96us/step - loss: 1.2729 - acc: 0.4767 - val_loss: 1.4433 - val_acc: 0.4628
Epoch 3/10
49158/49158 [==============================] - 5s 95us/step - loss: 1.2560 - acc: 0.4768 - val_loss: 1.2489 - val_acc: 0.4791
Epoch 4/10
49158/49158 [==============================] - 5s 94us/step - loss: 1.2458 - acc: 0.4795 - val_loss: 1.3517 - val_acc: 0.4628
Epoch 5/10
49158/49158 [==============================] - 5s 96us/step - loss: 1.2434 - acc: 0.4817 - val_loss: 1.3051 - val_acc: 0.4566
Epoch 6/10
49158/49158 [==============================] - 5s 95us/step - loss: 1.2399 - acc: 0.4803 - val_loss: 1.3090 - val_acc: 0.4571
Epoch 7/10
49158/49158 [==============================] - 5s 97us/step - loss: 1.2389 - acc: 0.4809 - val_loss: 1.2639 - val_acc

In [33]:
new_model.save('new_loss_model.h5')

In [45]:
def make_prediction(model, board):
    prediction_list = list(model.predict(np.array([board]))[0])
    move = prediction_list.index(max(prediction_list))
    print(prediction_list)
    print(move)

In [35]:
# Test 1
for model in [new_model, parial_model]:
    make_prediction(model, [0,0,0,
                            0,0,0,
                            0,0,0])
    print()

[0.08705829, 0.042095996, 0.120577954, 0.1116052, 0.09858884, 0.19317834, 0.15567848, 0.06502665, 0.1261903]
5

[0.0724203, 0.05704599, 0.11077164, 0.13728996, 0.117960274, 0.19007541, 0.105418645, 0.08204525, 0.12697251]
5



In [36]:
# Test 2
print('correct answer is 6\n')
for model in [new_model, parial_model]:
    make_prediction(model, [1,-1, 0,
                            1, 0, 0,
                            0,-1, 0])
    print()

correct answer is 6

[0.0016998232, 0.45132422, 5.340927e-08, 0.00026407052, 2.7811368e-14, 0.062088132, 5.277201e-05, 0.48454967, 2.1246635e-05]
7

[0.00055867643, 0.0043586534, 0.009743671, 0.0014634077, 0.09002632, 0.14640775, 0.63609505, 0.0051225056, 0.106223926]
6



In [37]:
# Test 3
print('correct answer is 8\n')
for model in [new_model, parial_model]:
    make_prediction(model, [1,-1, 0,
                            0, 1, 0,
                            0,-1, 0])
    print()

correct answer is 8

[1.065446e-05, 0.9762814, 2.7794428e-08, 0.012896196, 2.5951622e-15, 0.009994961, 1.2420514e-05, 0.0007658288, 3.855595e-05]
1

[0.00045785095, 0.0016771563, 0.04186555, 0.4657684, 0.00047322657, 0.06195486, 0.11757201, 0.00038255172, 0.30984846]
3



In [38]:
# Test 4
print('correct answer is 5 or 6\n')
for model in [new_model, parial_model]:
    make_prediction(model, [1,-1, 0,
                            1, 1, 0,
                            0,-1, -1])
    print()

correct answer is 5 or 6

[5.654078e-09, 3.341303e-09, 5.658157e-20, 7.554075e-11, 0.0, 0.9999999, 1.2858009e-13, 1.411965e-07, 9.425107e-16]
5

[1.2663373e-19, 5.5706685e-15, 1.267243e-16, 2.4310365e-12, 2.437692e-33, 0.99953187, 0.00046813147, 1.435266e-11, 9.830525e-16]
5



In [39]:
# Test 5
print('correct answer is 0\n')
for model in [new_model, parial_model]:
    make_prediction(model, [0,-1,  0,
                            1, 1, -1,
                            0,-1,  1])
    print()

correct answer is 0

[0.99401456, 1.4809096e-07, 3.2183128e-10, 6.462481e-08, 1.3331589e-18, 2.9747804e-05, 1.0059728e-05, 0.0059445333, 7.834826e-07]
0

[0.5510967, 3.0734547e-05, 0.01329813, 0.0022060194, 4.833137e-08, 0.00013696971, 0.43296203, 0.0002375819, 3.1798445e-05]
0



In [40]:
# Test 6
print('correct answer is 0\n')
for model in [new_model, parial_model]:
    make_prediction(model, [0,-1,  0,
                            1, 0, -1,
                            1,-1,  1])
    print()

correct answer is 0

[0.96695507, 5.9720486e-07, 1.1360268e-08, 1.7980565e-08, 2.0041508e-17, 7.36748e-07, 1.8301902e-08, 0.033043675, 1.3556401e-08]
0

[0.9193261, 0.00071931473, 0.030892884, 0.0071542994, 0.0013991677, 9.0813984e-07, 2.1822583e-05, 0.040448602, 3.6978963e-05]
0



[0.0724203, 0.05704599, 0.11077164, 0.13728996, 0.117960274, 0.19007541, 0.105418645, 0.08204525, 0.12697251]
5


In [42]:
make_prediction(parial_model, [-1,-1,1,1,0,-1,-1,1,1])

AttributeError: 'Sequential' object has no attribute 'index'

In [46]:
make_prediction(parial_model, [-1,-1, 0,
                                1, 0,-1,
                               -1, 1, 1])

[0.0, 0.0, 0.0, 0.9998913, 0.0, 5.231348e-07, 0.0, 0.00010825082, 0.0]
3


In [47]:
#prediction
[0.0, 0.0, 0.0, 
 0.9998913, 0.0, 5.231348e-07, 
 0.0, 0.00010825082, 0.0]

[0.0, 0.0, 0.0, 0.9998913, 0.0, 5.231348e-07, 0.0, 0.00010825082, 0.0]

In [2]:
def make_prediction(model, board):
    prediction_list = list(model.predict(np.array([board]))[0])
    move = prediction_list.index(max(prediction_list))
    print(prediction_list)
    print(move)

In [3]:
from keras.models import load_model
parial_model = load_model('smart_model_partial.h5')

from keras.models import load_model
pre_model = load_model('pre-trainied_loss_model.h5')

In [5]:
make_prediction(pre_model, [-1,-1, 0,
                                1, 0,-1,
                               -1, 1, 1])

[0.0, 0.0, 0.0, 0.99989116, 0.0, 5.231028e-07, 0.0, 0.0001082475, 0.0]
3


In [21]:
current_board = [[0,0,0],[0,0,0],[0,0,0]]

def board_to_prediction_list(board):
    vector_board = []
    for row in board:
        vector_board += row
    np_board = np.array([vector_board])

    return list(model.predict(np_board)[0])

from random import choice
def make_prediction(predicted_moves, available_moves):
    
    while sum(predicted_moves):
        print(predicted_moves)
        move = predicted_moves.index(max(predicted_moves))
        predicted_moves[move] = 0
        print(move)
        
        
        if move in available_moves:
            return move
        
    return choice(list(available_moves))


In [22]:
make_prediction([0.0, 0.0, 0.0, 0.99989116, 0.0, 5.231028e-07, 0.0, 0.0001082475, 0.0],{2,4})

[0.0, 0.0, 0.0, 0.99989116, 0.0, 5.231028e-07, 0.0, 0.0001082475, 0.0]
3
[0.0, 0.0, 0.0, 0, 0.0, 5.231028e-07, 0.0, 0.0001082475, 0.0]
7
[0.0, 0.0, 0.0, 0, 0.0, 5.231028e-07, 0.0, 0, 0.0]
5


2

In [11]:
[0,0].index(max([0,0]))

0